# Quick Fix for Open Notebook CLI Dependency Issues

This notebook provides a quick solution for the dependency issues with the Open Notebook CLI.

## The Problem

When trying to run commands using the `on_run.sh` script, you encounter this error:

```
gregory@macbook:~/localcode/open-notebook$ ./on_run.sh list-notebooks
Traceback (most recent call last):
  File "/app/open_notebook_cli.py", line 22, in <module>
    from open_notebook.domain.notebook import (
  File "/app/open_notebook/domain/notebook.py", line 4, in <module>
    from loguru import logger
ModuleNotFoundError: No module named 'loguru'
```

This indicates that the required Python dependencies are not installed in the Docker container.

## The Solution

I've updated the `on_run.sh` script to automatically check for and install missing dependencies before running any command.

The updated script does the following:

1. Checks if the `loguru` module can be imported in the container
2. If not, installs all the required dependencies using pip
3. Then proceeds to run the requested command

This makes the wrapper script much more reliable as it ensures dependencies are present before attempting to run any command.

In [ ]:
#!/bin/bash
# on_run.sh - Simple script to run Open Notebook CLI commands in the container
# Enhanced version that ensures dependencies are installed

CONTAINER_NAME="assistant-notebook-1"
CLI_PATH="/app/open_notebook_cli.py"

# Function to check if dependencies are installed
check_dependencies() {
  echo "Checking if required dependencies are installed..."
  
  # Test if loguru can be imported
  if ! docker exec $CONTAINER_NAME python -c "import loguru" 2>/dev/null; then
    echo "Installing missing dependencies..."
    # Install dependencies using pip
    docker exec $CONTAINER_NAME pip install loguru pydantic langchain langgraph typing_extensions humanize pyyaml
    echo "Dependencies installed."
  else
    echo "Dependencies already installed."
  fi
}

# Display help if no arguments provided
if [ $# -eq 0 ]; then
  echo "Usage: ./on_run.sh <command> [options]"
  echo "Example: ./on_run.sh list-notebooks"
  exit 1
fi

# Check and install dependencies before running command
check_dependencies

# Run the command in the container
echo "Running command: python $CLI_PATH $@"
docker exec -it "$CONTAINER_NAME" python "$CLI_PATH" "$@"

## How to Use the Updated Script

Make sure the script is executable:

```bash
chmod +x on_run.sh
```

Then use it as before:

```bash
./on_run.sh list-notebooks
./on_run.sh create-notebook "My Research" "Notes about my research"
```

The first time you run a command, it will check if dependencies are installed and install them if needed. Subsequent commands will run directly if dependencies are already installed.

## Why This Solution Works

This solution is straightforward and reliable because:

1. It checks for missing dependencies at runtime
2. It installs dependencies only when needed
3. It doesn't require modifying the container itself
4. It doesn't interfere with the container's normal operation

This approach is much simpler than attempting to diagnose complex Python environment issues within the container.

## If Problems Persist

If you still encounter issues after using the updated script, try:

1. Connecting directly to the container to diagnose the issue:
   ```bash
   docker exec -it assistant-notebook-1 bash
   ```

2. Installing the dependencies manually:
   ```bash
   docker exec -it assistant-notebook-1 pip install --force-reinstall loguru pydantic langchain langgraph
   ```

3. Running the more comprehensive setup script if needed:
   ```bash
   python open_notebook_container_setup.py
   ```

The last option will perform a more thorough diagnosis and fix of the container environment.